# Mpls Marriott Hotel Simulations
8760 rows per simulation<br>

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.

4. Use E+ template file to change the HVAC system. See Word doc that describes the process. Process all IDFs.

5. Rerun batch sims in EnergyPlus v9.5.

6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
# State Average
# comgas_cost = 0.60
# comelec_cost = 0.1043

# Provided by client
comgas_cost = 0.62915
comelec_cost = 0.127249

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.65

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MMBtu) 323.34 (lb/kbtu).32334 ####
#Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####

gas_co2 = 0.14566
elec_co2 = 0.2964

In [5]:
# Zone size for EUI and cost/sf

sf = 210

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory 

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/msp2/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/msp2/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
# all_ComElec

# 78840  rows, 55 columns

In [10]:
# list(all_ComElec.columns.values)

In [11]:
# Get wanted columns
all_energy = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)",
              "VAV SYS 1:Air System Fan Electricity Energy [J](Hourly) ",
              "MAIN CHILLER:Chiller Electricity Energy [J](Hourly)",
             "MAIN BOILER:Boiler NaturalGas Energy [J](Hourly)"]]

# all_energy

In [12]:
all_energy = all_energy.dropna().reset_index(drop=True)

#all_energy

In [13]:
# Export grouped file to csv
all_energy.to_csv("Scraper_Output/msp2_all.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [14]:
# Rename Columns
all_energy2 = all_energy.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "VAV SYS 1:Air System Fan Electricity Energy [J](Hourly) ":"FanEnergy[J](Hourly)",
                             "MAIN CHILLER:Chiller Electricity Energy [J](Hourly)":"CoolingEnergy[J](Hourly)",
                            "MAIN BOILER:Boiler NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

# all_energy2

In [15]:
list(all_energy2.columns.values)

['Scenario',
 'Date_Time',
 'Elevation',
 'GlazingSystem',
 'LightingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)',
 'HeatingEnergy[J](Hourly)']

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [16]:
# Copy dataframe to modify and leave original df intact
get_annual = all_energy2.copy(deep=True)

# get_annual

In [17]:
# list(get_annual.columns.values)

In [18]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual["LightingEnergy(kBtu)"] = (get_annual["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

# get_annual

In [19]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)",  "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

# get_annual

In [20]:
# Add energy totals
totalEnergy = get_annual["FanEnergy(kBtu)"] + get_annual["HeatingEnergy(kBtu)"] + get_annual["LightingEnergy(kBtu)"] + get_annual["CoolingEnergy(kBtu)"]
        
get_annual["AnnualEnergy(kBtu)"] = totalEnergy

# get_annual

-----

## Get Energy per sf 
Add EUI (site) column<br>


In [21]:
# Get data per sf (energy/sf)
get_annual["FanEnergy(kBtu/sf)"] = (get_annual["FanEnergy(kBtu)"]) / sf
get_annual["HeatingEnergy(kBtu/sf)"] = (get_annual["HeatingEnergy(kBtu)"]) / sf
get_annual["CoolingEnergy(kBtu/sf)"] = (get_annual["CoolingEnergy(kBtu)"]) / sf
get_annual["LightingEnergy(kBtu/sf)"] = (get_annual["LightingEnergy(kBtu)"]) / sf

In [22]:
# Establish EUI (energy/sf)
get_annual["EUI(kBtu/sf)"] = (get_annual["FanEnergy(kBtu)"] +
                                    get_annual["HeatingEnergy(kBtu)"] +
                                    get_annual["LightingEnergy(kBtu)"] +  
                                    get_annual["CoolingEnergy(kBtu)"]) / sf
                                                       
# get_annual

-----

## Get Carbon per sf 
Add carbon/sf column

In [23]:
# Do carbon calculation using variable (lb/sf)
get_annual["CO2_Electric(lb/sf)"] = ((get_annual["FanEnergy(kBtu)"] +
                                    get_annual["LightingEnergy(kBtu)"] +
                                    get_annual["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ComElec

In [24]:
get_annual["CO2_Gas(lb/sf)"] = ((get_annual["HeatingEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# get_annual

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [25]:
# Get annual costs
get_annual_costs = all_energy2.copy(deep=True)

In [26]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanCost($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs["HeatingCost($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs["CoolingCost($)"] = (get_annual_costs["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs["LightingCost($)"] = (get_annual_costs["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux * comelec_cost

#get_annual_costs

In [27]:
# Drop columns
get_annual_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs

In [28]:
# Add cost totals
totalCost = get_annual_costs["FanCost($)"] +  get_annual_costs["HeatingCost($)"]  + get_annual_costs["CoolingCost($)"] + get_annual_costs["LightingCost($)"]
        
get_annual_costs["AnnualCost($)"] = totalCost

# get_annual_costs

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>


In [29]:
# Get data per sf ($/sf)
get_annual_costs["FanCost($/sf)"] = (get_annual_costs["FanCost($)"]) / sf
get_annual_costs["HeatingCost($/sf)"] = (get_annual_costs["HeatingCost($)"]) / sf
get_annual_costs["CoolingCost($/sf)"] = (get_annual_costs["CoolingCost($)"]) / sf
get_annual_costs["LightingCost($/sf)"] = (get_annual_costs["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [30]:
# Establish costs ($/sf)
get_annual_costs["AnnualCost($/sf)"] = (get_annual_costs["FanCost($)"] +
                                    get_annual_costs["HeatingCost($)"] +  
                                    get_annual_costs["CoolingCost($)"] +  
                                    get_annual_costs["LightingCost($)"]) / sf
                                                       
# get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [31]:
# # Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

# allAnnualData

In [32]:
# list(allAnnualDataComElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [33]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualData.sum()

In [34]:
annualDataFinal = annualData.sum()

# annualDataFinal

In [35]:
# list(annualDataFinal.columns.values)

In [36]:
# Export grouped file to csv
annualDataFinal.to_csv("Scraper_Output/msp2_annual_com.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Export to CSV files


In [37]:
# Define the scenario for the existing condition.
baselineAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "Exist"]

# baselineAnnualData

In [38]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualData = baselineAnnualData.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'B_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'B_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingCost($)':'B_HeatingCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingCost($/sf)':'B_HeatingCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

# allBaselineAnnualDataComElec

In [39]:
# Reset index
allBaselineAnnualData.reset_index(inplace=True, drop=True)

# allBaselineAnnualData

In [40]:
# # Define the scenario for the baselines.

clearAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "Clear"]

# clearAnnualData

In [41]:
# Rename columns to indicate baseline to prepare for merge"

allClearAnnualData = clearAnnualData.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'FanEnergy(kBtu)':'C_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'C_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'C_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'C_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'C_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'C_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'C_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'C_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'C_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'C_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'C_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'C_CO2_Gas(lb/sf)',
 'FanCost($)':'C_FanCost($)',
 'HeatingCost($)':'C_HeatingCost($)',
 'CoolingCost($)':'C_CoolingCost($)',
 'LightingCost($)':'C_LightingCost($)',
 'AnnualCost($)':'C_AnnualCost($)',
 'FanCost($/sf)':'C_FanCost($/sf)',
 'HeatingCost($/sf)':'C_HeatingCost($/sf)',
 'CoolingCost($/sf)':'C_CoolingCost($/sf)',
 'LightingCost($/sf)':'C_LightingCost($/sf)',
 'AnnualCost($/sf)':'C_AnnualCost($/sf)'
                            })

# allClearAnnualData

In [42]:
# Reset index
allClearAnnualData.reset_index(inplace=True, drop=True)

# allClearAnnualDataComElec

In [43]:
# Define the scenario for the baselines.

loweAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "Lowe"]

# loweAnnualDataComElec

In [44]:
# Rename columns to indicate baseline to prepare for merge"

allLoweAnnualData = loweAnnualData.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'FanEnergy(kBtu)':'L_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'L_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'L_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'L_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'L_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'L_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'L_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'L_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'L_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'L_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'L_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'L_CO2_Gas(lb/sf)',
 'FanCost($)':'L_FanCost($)',
 'HeatingCost($)':'L_HeatingCost($)',
 'CoolingCost($)':'L_CoolingCost($)',
 'LightingCost($)':'L_LightingCost($)',
 'AnnualCost($)':'L_AnnualCost($)',
 'FanCost($/sf)':'L_FanCost($/sf)',
 'HeatingCost($/sf)':'L_HeatingCost($/sf)',
 'CoolingCost($/sf)':'L_CoolingCost($/sf)',
 'LightingCost($/sf)':'L_LightingCost($/sf)',
 'AnnualCost($/sf)':'L_AnnualCost($/sf)'
                            })

# allLoweAnnualDataComElec

In [45]:
#Reset index
allLoweAnnualData.reset_index(inplace=True, drop=True)

# allLoweAnnualDataComElec

---
## Merge the 3 dataframes to perform delta from existing calculation


In [46]:
# merge existing and clear
MergedAnnualData = pd.merge(allBaselineAnnualData, allClearAnnualData, left_index=True, right_index=True)

# merge the above with lowe
AllMergedAnnualData = pd.merge(MergedAnnualData, allLoweAnnualData, left_index=True, right_index=True)

# AllMergedAnnualData

In [47]:
# list(AllMergedAnnualData.columns.values)

In [48]:
# Deltas of exist - clear

AllMergedAnnualData["DeltaClear_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["C_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["C_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["C_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["C_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaClear_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["C_FanCost($)"]
AllMergedAnnualData["DeltaClear_HeatingCost($)"] = AllMergedAnnualData["B_HeatingCost($)"] - AllMergedAnnualData["C_HeatingCost($)"]
AllMergedAnnualData["DeltaClear_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["C_CoolingCost($)"]
AllMergedAnnualData["DeltaClear_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["C_LightingCost($)"]
AllMergedAnnualData["DeltaClear_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["C_AnnualCost($)"]
AllMergedAnnualData["DeltaClear_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["C_FanCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingCost($/sf)"] = AllMergedAnnualData["B_HeatingCost($/sf)"] - AllMergedAnnualData["C_HeatingCost($/sf)"]
AllMergedAnnualData["DeltaClear_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["C_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaClear_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["C_LightingCost($/sf)"]
AllMergedAnnualData["DeltaClear_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["C_AnnualCost($/sf)"]


In [49]:
# AllMergedAnnualData

In [50]:
# list(AllMergedAnnualData.columns.values)

In [51]:
# Deltas of exist - lowe

AllMergedAnnualData["DeltaLowe_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["L_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["L_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["L_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["L_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaLowe_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["L_FanCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingCost($)"] = AllMergedAnnualData["B_HeatingCost($)"] - AllMergedAnnualData["L_HeatingCost($)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["L_CoolingCost($)"]
AllMergedAnnualData["DeltaLowe_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["L_LightingCost($)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["L_AnnualCost($)"]
AllMergedAnnualData["DeltaLowe_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["L_FanCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingCost($/sf)"] = AllMergedAnnualData["B_HeatingCost($/sf)"] - AllMergedAnnualData["L_HeatingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["L_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["L_LightingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["L_AnnualCost($/sf)"]


In [52]:
# list(AllMergedAnnualData.columns.values)

In [53]:
# AllMergedAnnualData

---
## Group and Export Annual Delta Data
Group dataframe by scenario<br>
Export to file

In [54]:
# Group by scenario
AnnualDataDelta = AllMergedAnnualData.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=False)

# AnnualDataDelta.sum()

In [55]:
# list(AnnualDataDelta.sum().columns.values)

In [56]:
AnnualDataDelta = AnnualDataDelta.sum()

In [57]:
# Export grouped file to csv
AnnualDataDelta.to_csv("Scraper_Output/msp2_deltas_annual_com.csv", header=True, index=False)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [58]:
# Copy dataframe to modify and leave original df intact
get_peak = all_energy2.copy(deep=True)

# get_peak

In [59]:
# list(get_peak.columns.values)

In [60]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalPeakElec = (get_peak["FanEnergy[J](Hourly)"]  + get_peak["CoolingEnergy[J](Hourly)"] + get_peak["LightingEnergy[J](Hourly)"])
totalPeakGas =  get_peak["HeatingEnergy[J](Hourly)"]
        
get_peak["PeakElecEnergy[J](Hourly)"] = totalPeakElec
get_peak["PeakGasEnergy[J](Hourly)"] = totalPeakGas


get_peak["PeakElecEnergy(W)"] = (get_peak["PeakElecEnergy[J](Hourly)"] / 3600)
get_peak["PeakElecEnergy(W/sf)"] = (get_peak["PeakElecEnergy(W)"] / sf)


get_peak["PeakGasEnergy(W)"] = (get_peak["PeakGasEnergy[J](Hourly)"] / 3600)
get_peak["PeakGasEnergy(W/sf)"] = (get_peak["PeakGasEnergy(W)"] / sf)

# get_peak

In [61]:
# get_peak["PeakGasEnergy[J](Hourly)"].idxmax()

In [62]:
# get_peak["PeakElecEnergy[J](Hourly)"].idxmax()

In [63]:
# Get scenario
Eexist = get_peak[get_peak["Scenario"] == "Eexist"]
Eclear = get_peak[get_peak["Scenario"] == "Eclear"]
Elowe = get_peak[get_peak["Scenario"] == "Elowe"]
NWexist = get_peak[get_peak["Scenario"] == "NWexist"]
NWclear = get_peak[get_peak["Scenario"] == "NWclear"]
NWlowe = get_peak[get_peak["Scenario"] == "NWlowe"]
SWexist = get_peak[get_peak["Scenario"] == "SWexist"]
SWclear = get_peak[get_peak["Scenario"] == "SWclear"]
SWlowe = get_peak[get_peak["Scenario"] == "SWlowe"]


In [64]:
EexistPeakGas = pd.DataFrame(Eexist.loc[Eexist['HeatingEnergy[J](Hourly)'].idxmax()])
EexistPeakGas = EexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
EclearPeaKGas = pd.DataFrame(Eclear.loc[Eclear['HeatingEnergy[J](Hourly)'].idxmax()])
EclearPeaKGas = EclearPeaKGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
ElowePeakGas = pd.DataFrame(Elowe.loc[Elowe['HeatingEnergy[J](Hourly)'].idxmax()])
ElowePeakGas = ElowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWexistPeakGas = pd.DataFrame(NWexist.loc[NWexist['HeatingEnergy[J](Hourly)'].idxmax()])
NWexistPeakGas = NWexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWclearPeakGas = pd.DataFrame(NWclear.loc[NWclear['HeatingEnergy[J](Hourly)'].idxmax()])
NWclearPeakGas = NWclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWlowePeakGas = pd.DataFrame(NWlowe.loc[NWlowe['HeatingEnergy[J](Hourly)'].idxmax()])
NWlowePeakGas = NWlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWexistPeakGas = pd.DataFrame(SWexist.loc[SWexist['HeatingEnergy[J](Hourly)'].idxmax()])
SWexistPeakGas = SWexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWclearPeakGas = pd.DataFrame(SWclear.loc[SWclear['HeatingEnergy[J](Hourly)'].idxmax()])
SWclearPeakGas =  SWclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWlowePeakGas = pd.DataFrame(SWlowe.loc[SWlowe['HeatingEnergy[J](Hourly)'].idxmax()])
SWlowePeakGas = SWlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)

# EexistPeakGas

In [65]:
EexistPeakElec = pd.DataFrame(Eexist.loc[Eexist['CoolingEnergy[J](Hourly)'].idxmax()])
EexistPeakElec = EexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
EclearPeaKElec = pd.DataFrame(Eclear.loc[Eclear['CoolingEnergy[J](Hourly)'].idxmax()])
EclearPeaKElec = EclearPeaKElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
ElowePeakElec = pd.DataFrame(Elowe.loc[Elowe['CoolingEnergy[J](Hourly)'].idxmax()])
ElowePeakElec = ElowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWexistPeakElec = pd.DataFrame(NWexist.loc[NWexist['CoolingEnergy[J](Hourly)'].idxmax()])
NWexistPeakElec = NWexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWclearPeakElec = pd.DataFrame(NWclear.loc[NWclear['CoolingEnergy[J](Hourly)'].idxmax()])
NWclearPeakElec = NWclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWlowePeakElec = pd.DataFrame(NWlowe.loc[NWlowe['CoolingEnergy[J](Hourly)'].idxmax()])
NWlowePeakElec = NWlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWexistPeakElec = pd.DataFrame(SWexist.loc[SWexist['CoolingEnergy[J](Hourly)'].idxmax()])
SWexistPeakElec = SWexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWclearPeakElec = pd.DataFrame(SWclear.loc[SWclear['CoolingEnergy[J](Hourly)'].idxmax()])
SWclearPeakElec =  SWclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWlowePeakElec = pd.DataFrame(SWlowe.loc[SWlowe['CoolingEnergy[J](Hourly)'].idxmax()])
SWlowePeakElec = SWlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)

# EexistPeakElec

In [66]:
PeakDemandElec = EexistPeakElec.append([EclearPeaKElec,
ElowePeakElec,
NWexistPeakElec,
NWclearPeakElec,
NWlowePeakElec,
SWexistPeakElec,
SWclearPeakElec,
SWlowePeakElec
])

PeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Eexist,07/05 18:00:00,E,Exist,42336,804832,5.62961e+06,0,6.47678e+06,0,1799.1,8.56717,0,0
0,Eclear,07/05 18:00:00,E,Clear,42336,735788,5.26508e+06,0,6.0432e+06,0,1678.67,7.99365,0,0
0,Elowe,07/05 18:00:00,E,Lowe,42336,651649,4.77525e+06,0,5.46923e+06,0,1519.23,7.23444,0,0
0,NWexist,07/19 09:00:00,NW,Exist,66528,383893,2.97376e+06,0,3.42418e+06,0,951.162,4.52934,0,0
0,NWclear,07/03 10:00:00,NW,Clear,66528,277626,2.77057e+06,0,3.11473e+06,0,865.202,4.12001,0,0
0,NWlowe,07/06 09:00:00,NW,Lowe,66528,309995,2.47572e+06,0,2.85224e+06,0,792.29,3.77281,0,0
0,SWexist,08/02 10:00:00,SW,Exist,66528,499858,4.07679e+06,0,4.64318e+06,0,1289.77,6.14177,0,0
0,SWclear,08/02 10:00:00,SW,Clear,66528,460662,3.72882e+06,0,4.25601e+06,0,1182.22,5.62964,0,0
0,SWlowe,08/02 11:00:00,SW,Lowe,42336,416541,3.36278e+06,0,3.82166e+06,0,1061.57,5.0551,0,0


In [67]:
PeakDemandGas = EexistPeakGas.append([EclearPeaKGas,
ElowePeakGas,
NWexistPeakGas,
NWclearPeakGas,
NWlowePeakGas,
SWexistPeakGas,
SWclearPeakGas,
SWlowePeakGas
])

PeakDemandGas

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Eexist,01/03 09:00:00,E,Exist,66528,1.02675e+06,2.8297e+06,1.58691e+07,3.92298e+06,1.58691e+07,1089.72,5.18912,4408.08,20.9909
0,Eclear,01/03 08:00:00,E,Clear,84672,957034,2.55362e+06,1.30746e+07,3.59533e+06,1.30746e+07,998.702,4.75572,3631.84,17.2945
0,Elowe,01/03 09:00:00,E,Lowe,66528,857114,2.1775e+06,1.23522e+07,3.10114e+06,1.23522e+07,861.428,4.10204,3431.16,16.3389
0,NWexist,01/03 08:00:00,NW,Exist,84672,508136,1.21063e+06,1.14325e+07,1.80344e+06,1.14325e+07,500.956,2.3855,3175.7,15.1224
0,NWclear,01/03 08:00:00,NW,Clear,84672,466883,1.03385e+06,9.36137e+06,1.5854e+06,9.36137e+06,440.389,2.09709,2600.38,12.3828
0,NWlowe,01/03 08:00:00,NW,Lowe,84672,411781,928454,9.38247e+06,1.42491e+06,9.38247e+06,395.807,1.8848,2606.24,12.4107
0,SWexist,01/03 09:00:00,SW,Exist,66528,696296,1.89666e+06,1.26528e+07,2.65949e+06,1.26528e+07,738.747,3.51784,3514.67,16.7365
0,SWclear,01/03 09:00:00,SW,Clear,66528,630665,1.67497e+06,1.00918e+07,2.37216e+06,1.00918e+07,658.934,3.13778,2803.29,13.349
0,SWlowe,01/03 08:00:00,SW,Lowe,84672,564184,1.38251e+06,9.99339e+06,2.03137e+06,9.99339e+06,564.269,2.687,2775.94,13.2188


In [68]:
# # Export dataframe  to csv
# PeakDemandElec.to_csv("Scraper_Output/msp2_peakelec_com.csv", header=True, index=False)
# PeakDemandGas.to_csv("Scraper_Output/msp2_peakgas_com.csv", header=True, index=False)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Export to CSV files


In [69]:
# Define the scenario for the existing condition.
baselinePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Exist"]

# baselinePeakDemandElec

In [70]:
# list(baselinePeakDemandElec.columns.values)

In [71]:
# Rename columns to indicate baseline to prepare for merge"

allbaselinePeakDemandElec = baselinePeakDemandElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'B_HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'B_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'B_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'B_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'B_PeakElecEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'B_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'B_PeakElecEnergy(W/sf)'
                            })

allbaselinePeakDemandElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),B_PeakElecEnergy(W),B_PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Eexist,07/05 18:00:00,E,Exist,42336,804832,5.62961e+06,0,6.47678e+06,0,1799.1,8.56717,0,0
0,NWexist,07/19 09:00:00,NW,Exist,66528,383893,2.97376e+06,0,3.42418e+06,0,951.162,4.52934,0,0
0,SWexist,08/02 10:00:00,SW,Exist,66528,499858,4.07679e+06,0,4.64318e+06,0,1289.77,6.14177,0,0


In [72]:
#Reset index
allbaselinePeakDemandElec.reset_index(inplace=True, drop=True)

allbaselinePeakDemandElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),B_PeakElecEnergy(W),B_PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Eexist,07/05 18:00:00,E,Exist,42336,804832,5.62961e+06,0,6.47678e+06,0,1799.1,8.56717,0,0
1,NWexist,07/19 09:00:00,NW,Exist,66528,383893,2.97376e+06,0,3.42418e+06,0,951.162,4.52934,0,0
2,SWexist,08/02 10:00:00,SW,Exist,66528,499858,4.07679e+06,0,4.64318e+06,0,1289.77,6.14177,0,0


In [73]:
# Define the scenario for the clear panel condition.
clearPeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Clear"]

clearPeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Eclear,07/05 18:00:00,E,Clear,42336,735788,5.26508e+06,0,6.0432e+06,0,1678.67,7.99365,0,0
0,NWclear,07/03 10:00:00,NW,Clear,66528,277626,2.77057e+06,0,3.11473e+06,0,865.202,4.12001,0,0
0,SWclear,08/02 10:00:00,SW,Clear,66528,460662,3.72882e+06,0,4.25601e+06,0,1182.22,5.62964,0,0


In [74]:
# Rename columns to indicate baseline to prepare for merge"

allClearPeakDemandElec = clearPeakDemandElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'C_HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'C_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'C_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'C_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'C_PeakElecEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'C_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'C_PeakElecEnergy(W/sf)'
                            })

# allClearPeakDemandElec

In [75]:
#Reset index
allClearPeakDemandElec.reset_index(inplace=True, drop=True)

# allClearPeakDemandElec

In [76]:
# Define the scenario for the lowe panel condition.
lowePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Lowe"]

lowePeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Elowe,07/05 18:00:00,E,Lowe,42336,651649,4.77525e+06,0,5.46923e+06,0,1519.23,7.23444,0,0
0,NWlowe,07/06 09:00:00,NW,Lowe,66528,309995,2.47572e+06,0,2.85224e+06,0,792.29,3.77281,0,0
0,SWlowe,08/02 11:00:00,SW,Lowe,42336,416541,3.36278e+06,0,3.82166e+06,0,1061.57,5.0551,0,0


In [77]:
# Rename columns to indicate baseline to prepare for merge"

allLowePeakDemandElec = lowePeakDemandElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'L_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'L_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'L_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'L_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'L_PeakElecEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'L_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'L_PeakElecEnergy(W/sf)'
                            })

# allLowePeakDemandElec

In [78]:
# Reset index
allLowePeakDemandElec.reset_index(inplace=True, drop=True)

# allLowePeakDemandElec

---
## Merge the 3 dataframes to perform delta peak from existing calculation


In [79]:
# merge existing and clear
MergedAnnualPeakElecData = pd.merge(allbaselinePeakDemandElec, allClearPeakDemandElec, left_index=True, right_index=True)

MergedAnnualPeakElecData

# # #merge the above with lowe
AllMergedAnnualPeakData = pd.merge(MergedAnnualPeakElecData, allLowePeakDemandElec, left_index=True, right_index=True)

MergedAnnualPeakElecData

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly)_x,...,C_LightingEnergy[J](Hourly),C_FanEnergy[J](Hourly),C_CoolingEnergy[J](Hourly),C_HeatingEnergy[J](Hourly),C_PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly)_y,C_PeakElecEnergy(W),C_PeakElecEnergy(W/sf),PeakGasEnergy(W)_y,PeakGasEnergy(W/sf)_y
0,Eexist,07/05 18:00:00,E,Exist,42336,804832,5.62961e+06,0,6.47678e+06,0,...,42336,735788,5.26508e+06,0,6.0432e+06,0,1678.67,7.99365,0,0
1,NWexist,07/19 09:00:00,NW,Exist,66528,383893,2.97376e+06,0,3.42418e+06,0,...,66528,277626,2.77057e+06,0,3.11473e+06,0,865.202,4.12001,0,0
2,SWexist,08/02 10:00:00,SW,Exist,66528,499858,4.07679e+06,0,4.64318e+06,0,...,66528,460662,3.72882e+06,0,4.25601e+06,0,1182.22,5.62964,0,0


In [80]:
# list(AllMergedAnnualPeakData.columns.values)

In [81]:
#Deltas of exist - clear

MergedAnnualPeakElecData["DeltaClear_PeakElecEnergy(W)"] = AllMergedAnnualPeakData["B_PeakElecEnergy(W)"] - AllMergedAnnualPeakData["C_PeakElecEnergy(W)"]
MergedAnnualPeakElecData["DeltaClear_PeakElecEnergy(W/sf)"] = AllMergedAnnualPeakData["B_PeakElecEnergy(W/sf)"] - AllMergedAnnualPeakData["C_PeakElecEnergy(W/sf)"]

MergedAnnualPeakElecData
# AllMergedAnnualPeakData

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly)_x,...,C_CoolingEnergy[J](Hourly),C_HeatingEnergy[J](Hourly),C_PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly)_y,C_PeakElecEnergy(W),C_PeakElecEnergy(W/sf),PeakGasEnergy(W)_y,PeakGasEnergy(W/sf)_y,DeltaClear_PeakElecEnergy(W),DeltaClear_PeakElecEnergy(W/sf)
0,Eexist,07/05 18:00:00,E,Exist,42336,804832,5.62961e+06,0,6.47678e+06,0,...,5.26508e+06,0,6.0432e+06,0,1678.67,7.99365,0,0,120.438,0.573515
1,NWexist,07/19 09:00:00,NW,Exist,66528,383893,2.97376e+06,0,3.42418e+06,0,...,2.77057e+06,0,3.11473e+06,0,865.202,4.12001,0,0,85.9597,0.409332
2,SWexist,08/02 10:00:00,SW,Exist,66528,499858,4.07679e+06,0,4.64318e+06,0,...,3.72882e+06,0,4.25601e+06,0,1182.22,5.62964,0,0,107.547,0.512129


In [82]:
# Deltas of exist - lowe

MergedAnnualPeakElecData["DeltaLowe_PeakElecEnergy(W)"] = AllMergedAnnualPeakData["B_PeakElecEnergy(W)"] - AllMergedAnnualPeakData["L_PeakElecEnergy(W)"]
MergedAnnualPeakElecData["DeltaLowe_PeakElecEnergy(W/sf)"] = AllMergedAnnualPeakData["B_PeakElecEnergy(W/sf)"] - AllMergedAnnualPeakData["L_PeakElecEnergy(W/sf)"]

MergedAnnualPeakElecData

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly)_x,...,C_PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly)_y,C_PeakElecEnergy(W),C_PeakElecEnergy(W/sf),PeakGasEnergy(W)_y,PeakGasEnergy(W/sf)_y,DeltaClear_PeakElecEnergy(W),DeltaClear_PeakElecEnergy(W/sf),DeltaLowe_PeakElecEnergy(W),DeltaLowe_PeakElecEnergy(W/sf)
0,Eexist,07/05 18:00:00,E,Exist,42336,804832,5.62961e+06,0,6.47678e+06,0,...,6.0432e+06,0,1678.67,7.99365,0,0,120.438,0.573515,279.873,1.33273
1,NWexist,07/19 09:00:00,NW,Exist,66528,383893,2.97376e+06,0,3.42418e+06,0,...,3.11473e+06,0,865.202,4.12001,0,0,85.9597,0.409332,158.872,0.756533
2,SWexist,08/02 10:00:00,SW,Exist,66528,499858,4.07679e+06,0,4.64318e+06,0,...,4.25601e+06,0,1182.22,5.62964,0,0,107.547,0.512129,228.2,1.08667


In [83]:
list(MergedAnnualPeakElecData.columns.values)

['B_Scenario',
 'B_Date_Time',
 'B_Elevation',
 'B_GlazingSystem',
 'B_LightingEnergy[J](Hourly)',
 'B_FanEnergy[J](Hourly)',
 'B_CoolingEnergy[J](Hourly)',
 'B_HeatingEnergy[J](Hourly)',
 'B_PeakElecEnergy[J](Hourly)',
 'PeakGasEnergy[J](Hourly)_x',
 'B_PeakElecEnergy(W)',
 'B_PeakElecEnergy(W/sf)',
 'PeakGasEnergy(W)_x',
 'PeakGasEnergy(W/sf)_x',
 'C_Scenario',
 'C_Date_Time',
 'C_Elevation',
 'C_GlazingSystem',
 'C_LightingEnergy[J](Hourly)',
 'C_FanEnergy[J](Hourly)',
 'C_CoolingEnergy[J](Hourly)',
 'C_HeatingEnergy[J](Hourly)',
 'C_PeakElecEnergy[J](Hourly)',
 'PeakGasEnergy[J](Hourly)_y',
 'C_PeakElecEnergy(W)',
 'C_PeakElecEnergy(W/sf)',
 'PeakGasEnergy(W)_y',
 'PeakGasEnergy(W/sf)_y',
 'DeltaClear_PeakElecEnergy(W)',
 'DeltaClear_PeakElecEnergy(W/sf)',
 'DeltaLowe_PeakElecEnergy(W)',
 'DeltaLowe_PeakElecEnergy(W/sf)']

In [84]:
# Export grouped file to csv
MergedAnnualPeakElecData.to_csv("Scraper_Output/msp2_deltas_peakElec_com.csv", header=True, index=False)